In [1]:
from pprint import pprint
from pathlib import Path

from ReChisel.testcase import Testcase
from ReChisel.chisel_code import ChiselCode
from ReChisel.verifier import VerifyResult, verify


bmcase = Testcase(
    'Prob001_zero', 
    'benchmarks/VerilogEval_Prob001/Prob001_zero_spec.txt',
    'benchmarks/VerilogEval_Prob001/Prob001_zero_ref.sv',
    'benchmarks/VerilogEval_Prob001/Prob001_zero_tb.sv'
)
print(bmcase.specification)



I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - output zero

The module should always outputs a LOW.




In [2]:

from ReChisel.generator import Generator
from ReChisel.reviewer import Reviewer


generator = Generator(
    init_gen_system_prompt=Path('prompts/chisel_generation.txt').read_text(encoding='utf-8'),
    init_gen_model='gpt-4o',
    syntax_correction_system_prompt=Path('prompts/syntax_correction.txt').read_text(encoding='utf-8'),
    functionality_correction_system_prompt=Path('prompts/functionality_correction.txt').read_text(encoding='utf-8'),
    correction_model='gpt-4o',
    verbose=True
)
generator.testcase_prepare(bmcase, 'TopModule')

reviewer = Reviewer(
    sbt_system_prompt=Path('prompts/syntax_sbt_reflection.txt').read_text(encoding='utf-8'),
    iv_system_prompt=Path('prompts/syntax_iv_reflection.txt').read_text(encoding='utf-8'),
    functionality_system_prompt=Path('prompts/functionality_reflection.txt').read_text(encoding='utf-8'),
    model='gpt-4o',
    verbose=True
)


[GENERATOR] Preparing testcase for code generation.
[GENERATOR] Testcase:Testcase(prob_id=Prob001_zero, specification_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_spec.txt, reference_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_ref.sv, testbench_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_tb.sv)
[GENERATOR] Top module name: TopModule


/home/niujuxin/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_arn" in BedrockRerank has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Correct

In [3]:

correct_chisel_code = ChiselCode(
    f"```scala\n{Path('benchmarks/VerilogEval_Prob001/gen_correct.scala').read_text(encoding='utf-8')}\n```",
    'TopModule'
)

verify_result: VerifyResult = verify(
    correct_chisel_code, bmcase, 
    output_dir=Path('output/VerilogEval_Prob001/'),
    bm_type='verilog-eval',
    verbose=False
)

_d = verify_result.__dict__()
_d.pop('compiled_verilog_code', None)  # Remove verilog code for better readability
pprint(_d)

{'chisel_compile_to_verilog_success': True,
 'functionality_correct': True,
 'iv_cmd_exec_result': {'return_code': 0, 'stderr': '', 'stdout': ''},
 'run_verilog_sim_success': True,
 'sbt_cmd_exec_result': {'return_code': 0,
                         'stderr': '',
                         'stdout': '[success] Total time: 3 s, completed '
                                   '2025年7月4日 下午2:24:34'},
 'verilog_compile_success': True,
 'vvp_cmd_exec_result': {'return_code': 0,
                         'stderr': '',
                         'stdout': 'VCD info: dumpfile wave.vcd opened for '
                                   'output.\n'
                                   "Hint: Output 'zero' has no mismatches.\n"
                                   'Hint: Total mismatched samples is 0 out of '
                                   '20 samples\n'
                                   '\n'
                                   'Simulation finished at 102 ps\n'
                                   'Mismatche

In [4]:
def one_round(incorrect_chisel_code: ChiselCode):
    
    print('==' * 20, "Verification of Incorrect Code", '==' * 20)
    verify_result: VerifyResult = verify(
        incorrect_chisel_code, bmcase, 
        output_dir=Path('output/VerilogEval_Prob001/'),
        bm_type='verilog-eval',
        verbose=False
    )
    _d = verify_result.__dict__()
    _d.pop('compiled_verilog_code', None)  # Remove verilog code for better readability
    pprint(_d)
    print()

    print('==' * 20, "Review", '==' * 20)
    reviewer_response = reviewer(bmcase, verify_result, incorrect_chisel_code)
    print('-' * 20, "Response", '-' * 20)
    print(reviewer_response.content)
    print()

    print('==' * 20, "Correction Code Generation", '==' * 20)
    generation_response = generator.correction_generation(
        reviewer_response, 
        verify_result, 
        incorrect_chisel_code
    )
    new_code = generator.code_extract(generation_response)
    print('-' * 20, "Chisel Code", '-' * 20)
    print(new_code.raw)
    print()

    print('==' * 20, "Verification of New Code", '==' * 20)
    new_verify_result: VerifyResult = verify(
        new_code, bmcase, 
        output_dir=Path('output/VerilogEval_Prob001/'),
        bm_type='verilog-eval',
        verbose=False
    )
    _d = new_verify_result.__dict__()
    _d.pop('compiled_verilog_code', None)  # Remove verilog code for better readability
    pprint(_d)


## Chisel Compilation to Verilog (`sbt` Command) Fail

```scala
/**
 * TopModule
 * 
 * Outputs a constant LOW (0) on the 'zero' output port.
 * No clock or reset is required (combinational logic).
 *
 * NOTE:
 * This code is intentionally incorrect for the purpose of testing reflection.
 * The output 'zero' should be `false.B`, but it is set to `false`.
 * This code cannot be compiled by `sbt` command.
 */
class TopModule extends RawModule {
  val zero = IO(Output(Bool()))
  zero := false  // This should be `false.B`, but is set to `false`.
}
```


In [5]:

sbt_failed_chisel_code = ChiselCode(
    f"```scala\n{Path('benchmarks/VerilogEval_Prob001/gen_sbt_failed.scala').read_text(encoding='utf-8')}\n```",
    'TopModule'
)

one_round(sbt_failed_chisel_code)

======================================== Verification of Incorrect Code ========================================
{'chisel_compile_to_verilog_success': False,
 'functionality_correct': False,
 'iv_cmd_exec_result': None,
 'run_verilog_sim_success': None,
 'sbt_cmd_exec_result': {'return_code': 1,
                         'stderr': '',
                         'stdout': '[error] '
                                   '/home/niujuxin/github_repo/ReChisel/output/VerilogEval_Prob001/chisel/src/main/scala/Main.scala:10:11: '
                                   'type mismatch;\n'
                                   '[error]  found   : Boolean(false)\n'
                                   '[error]  required: chisel3.Data\n'
                                   '[error]   zero := false  \n'
                                   '[error]           ^\n'
                                   '[error] one error found\n'
                                   '[error] (Compile / compileIncremental) '
               

## Verilog Compilation (`iverilog` Command) Fail

```scala
/**
 * TopModule
 * 
 * Outputs a constant LOW (0) on the 'zero' output port.
 * No clock or reset is required (combinational logic).
 * 
 * NOTE:
 * This code is intentionally incorrect for the purpose of testing reflection.
 * The output 'zero_1' should be named `zero` to match the testbench.
 * This code can be compiled by `sbt` command but the generated Verilog code is incompatible with testbench code.
 * The `iverilog` command will fail.
 */
class TopModule extends RawModule {
  val zero_1 = IO(Output(Bool())) // Signal should be named `zero` to match the testbench.
  zero_1 := false.B
}
```


In [6]:

iv_failed_chisel_code = ChiselCode(
    f"```scala\n{Path('benchmarks/VerilogEval_Prob001/gen_iv_failed.scala').read_text(encoding='utf-8')}\n```",
    'TopModule'
)

one_round(iv_failed_chisel_code)

======================================== Verification of Incorrect Code ========================================
{'chisel_compile_to_verilog_success': True,
 'functionality_correct': False,
 'iv_cmd_exec_result': {'return_code': 1,
                        'stderr': "Prob001_zero_tb.sv:75: error: port ``zero'' "
                                  'is not a port of top_module1.\n'
                                  '1 error(s) during elaboration.\n',
                        'stdout': ''},
 'run_verilog_sim_success': None,
 'sbt_cmd_exec_result': {'return_code': 0,
                         'stderr': '',
                         'stdout': '[success] Total time: 3 s, completed '
                                   '2025年7月4日 下午2:25:00'},
 'verilog_compile_success': False,
 'vvp_cmd_exec_result': None}

======================================== Review ========================================
[REVIEWER] Preparing messages for reflection.
[REVIEWER] Adopting IV reflection system prompt.
[REVIEWER]

## Functionality Incorrect

```scala
/**
 * TopModule
 * 
 * Outputs a constant LOW (0) on the 'zero' output port.
 * No clock or reset is required (combinational logic).
 *
 * NOTE:
 * This code is intentionally incorrect for the purpose of testing reflection.
 * The output 'zero' should be false.B, but it is set to true.B.
 * This code is completely syntactically correct and the `sbt` and `iverilog` commands can compile it,
 * but the functionality is incorrect.
 */
class TopModule extends RawModule {
  val zero = IO(Output(Bool()))
  zero := true.B // This should be false.B, but is set to true.B for testing purposes.
}
```

In [7]:

func_failed_chisel_code = ChiselCode(
    f"```scala\n{Path('benchmarks/VerilogEval_Prob001/gen_func_failed.scala').read_text(encoding='utf-8')}\n```",
    'TopModule'
)

one_round(func_failed_chisel_code)

======================================== Verification of Incorrect Code ========================================
{'chisel_compile_to_verilog_success': True,
 'functionality_correct': False,
 'iv_cmd_exec_result': {'return_code': 0, 'stderr': '', 'stdout': ''},
 'run_verilog_sim_success': True,
 'sbt_cmd_exec_result': {'return_code': 0,
                         'stderr': '',
                         'stdout': '[success] Total time: 3 s, completed '
                                   '2025年7月4日 下午2:25:19'},
 'verilog_compile_success': True,
 'vvp_cmd_exec_result': {'return_code': 0,
                         'stderr': '',
                         'stdout': 'VCD info: dumpfile wave.vcd opened for '
                                   'output.\n'
                                   "Hint: Output 'zero' has 20 mismatches. "
                                   'First mismatch occurred at time 5.\n'
                                   'Hint: Total mismatched samples is 20 out '
                   